In [1]:
%autosave 0

Autosave disabled


# 推送天气预报

- 微信推给联系人
- 企业微信机器人推送
- 每天定时发送

### 使用天气api
+ https://www.tianqiapi.com/api/?version=v1&city=北京
+ https://www.tianqiapi.com/api/?version=v1&cityid=101190101

In [2]:
# -*- coding: utf-8 -*- 
from wxpy import *
import requests
from datetime import datetime
import time
import schedule
from apscheduler.schedulers.blocking import BlockingScheduler
import requests

In [3]:
bot = Bot(cache_path=True)

In [6]:
bot2 = Bot(cache_path=True)

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 夏沫秋雨


In [28]:
def weather_friend():
    my_friends = []
    my_friends.append(bot2.friends().search('Elsa')[0])
    return my_friends

def send_weather():
    my_friends = weather_friend()
    for friend in my_friends:
        friend.send(getWeather(friend.city))
        print()
        bot2.file_helper.send('天气推送完毕')
        
def getWeather(location):
    url = 'https://www.tianqiapi.com/api/?version=v1&city='+location
    response = requests.get(url)
    result = response.json()
    s1 = result['data'][0]
    s2 = s1['date'] + ' ' + s1['week'] + ' ' + s1['wea'] + ' ' + s1['tem1'] + '~' + s1['tem2'] + ' ' + '空气质量:' + s1['air_level'] + '\n'
    s3 = s1['index']
    s_advice = []
    i = 0
    for s4 in s3:
        if i != 1: 
            d = s4['title'] + ':' + s4['level'] + '(' + s4['desc'] + ')' + '\n'
            s2 += d
        i += 1
    return s2

In [23]:
def print_getWeather(location):
    url = 'https://www.tianqiapi.com/api/?version=v1&city='+location
    print("url:")
    print(url)
    print("=======================================================")
    response = requests.get(url)
    result = response.json()
    print("result:")
    print(result)
    s1 = result['data'][0]
    s2 = s1['date'] + ' ' + s1['week'] + ' ' + s1['wea'] + ' ' + s1['tem1'] + '~' + s1['tem2'] + ' ' + '空气质量:' + s1['air_level'] + '\n'
    s3 = s1['index']
    s_advice = []
    i = 0
    for s4 in s3:
        if i != 1: 
            d = s4['title'] + ':' + s4['level'] + '(' + s4['desc'] + ')' + '\n'
            s2 += d
        i += 1
    return s2

In [24]:
print_getWeather("南京")

url:
https://www.tianqiapi.com/api/?version=v1&city=南京
result:
{'cityid': '101190101', 'update_time': '2019-08-19 07:30:00', 'city': '南京', 'cityEn': 'nanjing', 'country': '中国', 'countryEn': 'China', 'data': [{'day': '19日（今天）', 'date': '2019-08-19', 'week': '星期一', 'wea': '多云', 'wea_img': 'yun', 'air': 54, 'humidity': 82, 'air_level': '良', 'air_tips': '空气好，可以外出活动，除极少数对污染物特别敏感的人群以外，对公众没有危害！', 'alarm': {'alarm_type': '', 'alarm_level': '', 'alarm_content': ''}, 'tem1': '32℃', 'tem2': '23℃', 'tem': '26℃', 'win': ['东风', '东风'], 'win_speed': '<3级', 'hours': [{'day': '19日08时', 'wea': '晴', 'tem': '26℃', 'win': '东风', 'win_speed': '<3级'}, {'day': '19日11时', 'wea': '多云', 'tem': '30℃', 'win': '东风', 'win_speed': '<3级'}, {'day': '19日14时', 'wea': '多云', 'tem': '31℃', 'win': '东风', 'win_speed': '<3级'}, {'day': '19日17时', 'wea': '多云', 'tem': '31℃', 'win': '东风', 'win_speed': '<3级'}, {'day': '19日20时', 'wea': '多云', 'tem': '26℃', 'win': '东风', 'win_speed': '<3级'}, {'day': '19日23时', 'wea': '多云', 'tem': '25℃', 'win

'2019-08-19 星期一 多云 32℃~23℃ 空气质量:良\n紫外线指数:中等(涂擦SPF大于15、PA+防晒护肤品。)\n健臻·血糖指数:较易波动(血糖较易波动，注意监测。)\n穿衣指数:炎热(建议穿短衫、短裤等清凉夏季服装。)\n洗车指数:较适宜(无雨且风力较小，易保持清洁度。)\n空气污染扩散指数:中(易感人群应适当减少室外活动。)\n'

In [46]:
def test_robot(str_weather):
    headers = {"Content-Type":"text/plain"}
    s = str_weather
#     s = "hello"
    
    data = {
        "msgtype":"text",
        "text":{
            "content":s,
            "mentioned_list":["qufang","duanxuezhen","@all"]
        }
    }
    r = requests.post(
        url='https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=18edab2b-6276-402e-9b00-ed5c38d204f3',
        headers=headers,
        json=data)
    print(r.text)

In [47]:
test_robot(getWeather("北京"))

{"errcode":0,"errmsg":"ok"}


LOG OUT!


## 每天定时发送

In [ ]:
print('star')
sched = BlockingScheduler()
sched.add_job(send_weather,
             'cron',
             month='1-12',
             day='1-31',
             hour=11,
             minute=5)
sched.start()